In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.layers import LSTM

warnings.filterwarnings(action='ignore')

heart_data = pd.read_csv('./data/train.csv')
heart_data.head()

2021-12-22 07:06:46.555983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 07:06:46.556333: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,1,53,1,2,130,197,1,0,152,0,1.2,0,0,2,1
1,2,52,1,3,152,298,1,1,178,0,1.2,1,0,3,1
2,3,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0
3,4,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
4,5,35,1,1,122,192,0,1,174,0,0.0,2,0,2,1


In [3]:
def check_missing_col(dataframe):
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'총 {missing_values}개의 결측치가 존재합니다.')

    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')

check_missing_col(heart_data)

결측치가 존재하지 않습니다


# Labeled encoding data features
- sex
    - female = 0, male = 1
- cp: 가슴 통증(chest pain) 종류 
  - 0: asymptomatic 무증상
  - 1: atypical angina 일반적이지 않은 협심증
  - 2: non-anginal pain 협심증이 아닌 통증
  - 3: typical angina 일반적인 협심증
- fbs: (fasting blood sugar) 공복 중 혈당
    - 120 mg/dl 이하일 시 = 0
    - 120 mg/dl 초과일 시 = 1
- restecg: (resting electrocardiographic) 휴식 중 심전도 결과 
  - 0: showing probable or definite left ventricular hypertrophy by Estes' criteria
  - 1: 정상
  - 2: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
- exang: (exercise induced angina) 활동으로 인한  협심증 여부
    - 없음 = 0
    - 있음 = 1
- slope: (the slope of the peak exercise ST segment) 활동 ST 분절 피크의 기울기
  - 0: downsloping 하강
  - 1: flat 평탄
  - 2: upsloping 상승
- ca: number of major vessels colored by flouroscopy 형광 투시로 확인된 주요 혈관 수 (0~3 개) 
  - Null 값은 숫자 4로 인코딩됨
- thal: thalassemia 지중해빈혈 여부
  - 0: Null 
  - 1: normal 정상
  - 2: fixed defect 고정 결함
  - 3: reversable defect 가역 결함
- target: 심장 질환 진단 여부
  - 0: < 50% diameter narrowing
  - 1: > 50% diameter narrowing

In [15]:
#process label features to one-hot encoding
proc_data = heart_data
proc_data = pd.get_dummies(proc_data, columns= ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'])
proc_data.head()

,id,age,trestbps,chol,thalach,oldpeak,target,sex_0,sex_1,cp_0,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,1,53,130,197,152,1.2,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,2,52,152,298,178,1.2,1,0,1,0,...,1,0,1,0,0,0,0,0,0,1
2,3,54,192,283,195,0.0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
3,4,45,138,236,152,0.2,1,1,0,1,...,1,0,1,0,0,0,0,0,1,0
4,5,35,122,192,174,0.0,1,0,1,0,...,0,1,1,0,0,0,0,0,1,0


In [ ]:
#classification learning model setting